In [ ]:
!pip install datasets transformers scikit-learn pandas


In [ ]:
from datasets import load_dataset
import pandas as pd

# Load dataset
dataset = load_dataset("banking77")

# Convert train and test to pandas
df_train = pd.DataFrame(dataset['train'])
df_test = pd.DataFrame(dataset['test'])

# Quick look
print("Training set size:", len(df_train))
print("Test set size:", len(df_test))
print(df_train.head())

In [ ]:
# Select features and target
X_train = df_train['text']
y_train = df_train['label']

X_test = df_test['text']
y_test = df_test['label']


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Fit on training data and transform
X_train_vec = vectorizer.fit_transform(X_train)

# Transform test data
X_test_vec = vectorizer.transform(X_test)


In [ ]:
from sklearn.linear_model import LogisticRegression

# Create the model
model = LogisticRegression(max_iter=300)

# Train on vectorized training data
model.fit(X_train_vec, y_train)


In [ ]:
from sklearn.metrics import classification_report

# Predict on test data
y_pred = model.predict(X_test_vec)

# Show performance metrics
print(classification_report(y_test, y_pred))


In [ ]:
from transformers import pipeline

# Load sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis")

In [ ]:
sample_ticket = "My card got blocked and I can't make a payment"

# Get sentiment
sentiment_result = sentiment_analyzer(sample_ticket)[0]
print(sentiment_result)

In [ ]:
sample_ticket = "My experience is good with the card usage"

# Get sentiment
sentiment_result = sentiment_analyzer(sample_ticket)[0]
print(sentiment_result)

In [ ]:
from transformers import pipeline

# Load text generation model
response_generator = pipeline("text2text-generation", model="t5-small")


In [ ]:
def generate_response(ticket_text, category_name, sentiment_label):
    if sentiment_label == "NEGATIVE":
        response = f"Thank you for contacting us regarding '{category_name}'. We understand your concern. Our team will assist you shortly. (Customer mood: {sentiment_label})"
    elif sentiment_label == "NEUTRAL":
        response = f"We have received your message regarding '{category_name}'. Thank you for reaching out. (Customer mood: {sentiment_label})"
    elif sentiment_label == "POSITIVE":
        response = f"Thank you for your feedback regarding '{category_name}'! We're glad everything is going well. (Customer mood: {sentiment_label})"
    else:
        response = f"Thank you for contacting us regarding '{category_name}'. (Customer mood: {sentiment_label})"
    return response


In [ ]:
label_map = dict(zip(range(len(dataset['train'].features['label'].names)),
                     dataset['train'].features['label'].names))

In [ ]:
def support_assistant(ticket_text):
    # 1️⃣ Predict category
    category_num= model.predict(vectorizer.transform([ticket_text]))[0]

    # Map to category name
    category_name = label_map[category_num]

    # 2️⃣ Predict sentiment
    sentiment_result = sentiment_analyzer(ticket_text)[0]
    sentiment_label = sentiment_result['label']

    # 3️⃣ Generate auto-response (template-based)
    response = generate_response(ticket_text, category_name, sentiment_label)

    # Return all results
    return {
        "Ticket Text": ticket_text,
        "Predicted Category": category_name,
        "Sentiment": sentiment_label,
        "AI Response": response
    }


In [ ]:
ticket = "My card got blocked and I can't make a payment"
result = support_assistant(ticket)
print(result)

In [ ]:
ticket = "Thank you for resolving my account issue so quickly!"
result = support_assistant(ticket)
print(result)

In [ ]:
while True:
    ticket = input("Enter a customer ticket (or type 'exit' to quit): ")
    if ticket.lower() == "exit":
        print("Exiting support assistant.")
        break
    result = support_assistant(ticket)
    print("\nPredicted Category:", result['Predicted Category'])
    print("Sentiment:", result['Sentiment'])
    print("AI Response:", result['AI Response'])
    print("\n" + "-"*50 + "\n")